In [ ]:
# Importing pre-requisite libraries for the project

import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os
import PyPDF2, io

from dotenv import load_dotenv

# Setting up environment variables
load_dotenv()

In [ ]:
# Initiate Selenium Framework (Chromium Headless Browser)

def initialize_chromium():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-infobars")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-extensions")
    options.add_argument("--headless") # If the user wants to use Chromium with a GUI, comment this line.
    global driver; driver = webdriver.Chrome(options = options)


# Release Selenium Framework (Chromium Headless Browser)

def release_chromium():
    driver.close()

In [ ]:
# Extracting the Regular Meetings data to a DataFrame

def meetings_extraction():
  global current_year; current_year = 2023
  meeting_no = 1
  reg_meets_all = []
  reg_meets = []
  reg_meets_cancelled = []
  reg_meets_without_verbals = []

  while current_year > 2010:
    url = f'https://ottawapoliceboard.ca/opsb-cspo/{current_year}-meetings.html'

    driver.get(url)
    page_src = driver.page_source
    soup = BeautifulSoup(page_src, 'html.parser')
    meetings_list = soup.find('table').find('tbody').find_all('tr')


    # Fetch only regular meetings information from the list of all meetings
    for index, tr in enumerate(reversed(meetings_list)):
        td = tr.find_all('td')
        if td[3].get_text().strip() == 'Regular Meeting':
          if current_year < 2013: date = f'{td[0].get_text().strip()}, {current_year}'
          else: date = td[0].get_text().strip()

          reg_meets_all.append({
              'Meeting #': meeting_no,
              'Date': date,
              'Location': td[1].get_text().strip(),
              'Time': td[2].get_text().strip(),
              'Meeting Type': td[3].get_text().strip(),
              'Meeting Page': td[0].find('a').get('href'),
              'Chief Verbal Report Present': '',
              'Verbal Report File URL': ''
            })
          meeting_no = meeting_no + 1
    print(f'Regular Meetings of Year {current_year} processed.')
    current_year = current_year - 1
  reg_meets_all_df = pd.DataFrame(reg_meets_all)


# Remove Cancelled Regular Meetings
  for x in reg_meets_all:
    if x['Location'] == 'CANCELLED':
      reg_meets_cancelled.append(x)
    else:
      reg_meets.append(x)
  reg_meets_cancelled_df = pd.DataFrame(reg_meets_cancelled)

  for meet in reg_meets:
    url = meet['Meeting Page']
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri = urlparse(url))
    driver.get(url)
    page_src = driver.page_source
    soup = BeautifulSoup(page_src, 'html.parser')


    # Finds all the reports based on different variations of "Chief's Verbal Report" taken into consideration
    elems = soup.find_all(text = lambda t: t and any(x in t for x in ["Chief’s verbal report", "Chief's verbal report", "CHIEF’S VERBAL REPORT", "CHIEF'S VERBAL REPORT"]))
    for el in elems:
      if ((el.find_parent()).find_parent()).find_parent().find_all('a', href = lambda href: href and "filestream" in href):
        urls = ((el.find_parent()).find_parent()).find_parent().find_all('a', href = lambda href: href and "filestream" in href)
        if url:
          for a in urls:
              verbal_report_url = f"{domain}{a['href']}"
              meet.update({'Chief Verbal Report Present': 'Yes'})
              meet.update({'Verbal Report File URL': verbal_report_url})
        else:
          meet.update({'Chief Verbal Report Present': 'No'})
          meet.update({'Verbal Report File URL': '-'})
      elif (((el.find_parent()).find_parent()).find_parent()).find_parent().find_all('a', href = lambda href: href and "filestream" in href):
        urls = (((el.find_parent()).find_parent()).find_parent()).find_parent().find_all('a', href = lambda href: href and "filestream" in href)
        if urls:
          for a in urls:
            if "filestream" in a['href']:
              verbal_report_url = f"{domain}{a['href']}"
              meet.update({'Chief Verbal Report Present': 'Yes'})
              meet.update({'Verbal Report File URL': verbal_report_url})
            else:
              meet.update({'Chief Verbal Report Present': 'No'})
              meet.update({'Verbal Report File URL': '-'})
        else:
          meet.update({'Chief Verbal Report Present': 'No'})
          meet.update({'Verbal Report File URL': '-'})


  reg_meets_df = pd.DataFrame(reg_meets)
  export_to_excel(reg_meets_df)


# Export the DataFrame to an Excel Sheet

def export_to_excel(df):
  df.to_excel('Final regular_meetings_output.xlsx', sheet_name = 'Regular Meetings')

In [ ]:
# Function to send an E-Mail to the recipient address entered in the input box along with the regular_meetings excel file attachment

def send_mail():
    recipient_mail = input("Enter an E-mail address to receive missing chief's verbal list")
    to_mail = recipient_mail
    gmail_usr = os.getenv('GMAIL_USR')
    gmail_pwd = os.getenv('GMAIL_PWD')

    msg = MIMEMultipart()
    msg['Subject'] = "Missing Chief's Verbals List"
    msg['From'] = gmail_usr
    msg['To'] = to_mail

    msgText = MIMEText("\n This mail consists of regular meetings list where Chief's Verbal document is missing \n p.f.a. \n", 'html')
    msg.attach(msgText)

    filename = "Final regular_meetings_output.xlsx"
    xlsx = MIMEApplication(open(filename, 'rb').read())
    xlsx.add_header('Content-Disposition', 'attachment', filename = 'Final regular_meetings_output.xlsx')
    msg.attach(xlsx)

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as smtpserver:
            smtpserver.ehlo()
            smtpserver.starttls()
            smtpserver.login(gmail_usr, gmail_pwd)
            smtpserver.sendmail(f'{gmail_usr}<Team-8 SMTP Mail Client>', to_mail, msg.as_string())
            smtpserver.quit()
    except Exception as e:
        print(e)

    print ('email sent!')

In [ ]:
# Function to go through each Chief Verbal Document URL and extract each sentence to an Excel File

def extract_text_from_documents():
    global crimes_data_df
    crimes_data_df = pd.DataFrame()
    
    # Read the Excel file
    excel_file = 'Final regular_meetings_output.xlsx'
    reports_df = pd.read_excel(excel_file)

    # Assigning the column containing PDF links named as 'pdf_urls'
    reports_df = reports_df.dropna()
    pdf_urls = reports_df[['Date', 'Verbal Report File URL']]

    # Read each PDF
    for index, row in pdf_urls.iterrows():
        resp = requests.get(row['Verbal Report File URL'])
        with io.BytesIO(resp.content) as file:
            # Create a PDF object
            pdf = PyPDF2.PdfReader(file)
            
            # Initialize a variable to store the extracted text
            global corpus
            corpus = []
            page_text = ""
            
            # Extract the text from each page of the PDF. We have only one page.
            for page in pdf.pages:
                page_text += page.extract_text()
            
            page_text = page_text.split('.')
            for sentence in page_text:
                sentence = sentence.lower()
                sentence = re.sub("&lt;/?.*&gt;", "&lt;&gt; ", sentence)
                sentence = re.sub(r'\s{2,}', ' ', sentence)
                sentence = sentence.strip()
                if len(sentence.split(' ')) >= 4:
                    corpus.append(sentence)
                else:
                    continue
        
        print("link started: ", index, row['Verbal Report File URL'])
        convert_to_df(corpus, row['Date'])
        crimes_data_df = crimes_data_df.append(crime_df)
        print("link over: ", index, row['Verbal Report File URL'])
        print(crimes_data_df)

    export_excel()
    print("Crime Analysis Completed! Please check the output file: 'Final all_extracted_sentences.xlsx' for the output.")
 
# Function to convert all of the corpus sentences to a singular dataframe
def convert_to_df(corpus, date):
    global crime_df
    crime_df = pd.DataFrame({'sentences': corpus})
    crime_df.insert(0, 'Date', date)
    
# Export the modified DataFrame
def export_excel():
    print(crimes_data_df)
    output_path = 'Final all_extracted_sentences.xlsx'
    crimes_data_df.to_excel(output_path, index = False)

In [ ]:
# Execution cell with all essential function run in one-go

initialize_chromium()
meetings_extraction()
release_chromium()
send_mail()
extract_text_from_documents()